In [5]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.benchmark import local_get_args
from examples.train_and_visu_non_recurrent import evaluate_config,train_the_config,get_ds
from high_level_DL_method import load_optimizer_and_scheduler
from dl_models.full_model import full_model
from trainer import Trainer
from constants.config_by_datasets import dic_config

# Init:
#['subway_indiv','tramway_indiv','bus_indiv','velov','criter']
target_data = 'subway_in' #'subway_in'  # PeMS03 # PeMS04 # PeMS07 # PeMS08 # METR_LA 
dataset_names = ['subway_in','subway_out'] #['PeMS03'] #['subway_in'] ['subway_in','subway_indiv'] #["subway_in","subway_out"] # ['subway_in','netmob_POIs_per_station'],["subway_in","subway_out"],["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station']#['subway_in','subway_indiv'] # ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN' # 'STGCN', 'ASTGCN'
station = ['BEL']# ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'target_data': target_data, 
                'freq': '15min', #'15min', # '5min'
                'use_target_as_context': False,
                'data_augmentation': False,
                'step_ahead':4,
        
                'epochs' : 100, #100

                'lr': 0.00105, # 5e-5,# 4e-4,
                'weight_decay': 0.0188896655584368, # 0.05,
                'dropout': 0.271795372610271, # 0.15,
                'Kt': 2,
                'stblock_num': 3,
                'gso_type': 'sym_renorm_adj',
                'temporal_h_dim': 256,
                'spatial_h_dim': 32,
                'output_h_dim': 16,
                'scheduler': True,  # None
                'torch_scheduler_milestone': 28.0, #5,
                'torch_scheduler_gamma': 0.9958348861339396, # 0.997,
                'torch_scheduler_lr_start_factor': 0.8809942312067847, # 1,


                #'set_spatial_units':  station,   
                'adj_type':'corr',
                'threshold': 0.7,
                'learnable_adj_matrix' : False, # True                              # EXIST ONLY IF MODEL = STGCN
                'graph_conv_type': 'graph_conv', # 'cheb_graph_conv', 'graph_conv'  # EXIST ONLY IF MODEL = STGCN
                'learnable_adj_top_k': 10,                                          # EXIST ONLY IF MODEL = STGCN
                'learnable_adj_embd_dim': 16,                                       # EXIST ONLY IF MODEL = STGCN  

                'stacked_contextual': True, # True # False
                'temporal_graph_transformer_encoder': False,
                'compute_node_attr_with_attn' : False, # True ??

                #
                #'graph_conv_type': 'graph_conv', # 'cheb_graph_conv', 'graph_conv'
                #'learnable_adj_top_k': 10,
                #'learnable_adj_embd_dim': 16, 
                ### ========

                ### Temporal Graph Transfermer Encoder parametrs:
                #'TGE_num_layers' : 4, #2
                #'TGE_num_heads' :  1, #IMPOSSIBLE > 1 CAR DOIT DIVISER L = 7
                #'TGE_FC_hdim' :  32, #32

                ### Netmob Parametrs: 
                #'NetMob_only_epsilon': True,    # True # False
                #'NetMob_selected_apps': ['Apple_iMessage','Web_Ads'],# ['Apple_iMessage','Web_Ads'], #,'Deezer','WhatsApp','Twitter'] #['Google_Maps']# ['Instagram','Google_Maps','Twitter']
                #'NetMob_transfer_mode' :  ['DL'], #,'UL'] # ['DL'] # ['UL'] #['DL','UL']
                #'NetMob_selected_tags' : ['station_epsilon100'],#['iris','stadium','station','university']#['park','stadium','university','station','shop','nightclub','parkings','theatre','iris','transit','public_transport']
                #'NetMob_expanded' : '', # '' # '_expanded

                ### Compute node with attention parameters: 
                #'vision_num_heads':6,
                #"vision_grn_out_dim":48,
                #'vision_model_name': 'VariableSelectionNetwork',
                #'vision_concatenation_early':True,   
                #'vision_concatenation_late':True,
                           }

if target_data in dic_config.keys():
    modification.update(dic_config[target_data])
# ...
#1038945

# Training and visu: 
args_init = local_get_args(model_name,
                    args_init = None,
                    dataset_names=dataset_names,
                    dataset_for_coverage=dataset_for_coverage,
                    modification = modification)

training_mode_to_visualise = ['test'] # ['test','valid','train']

if False:
    ds,args,trial_id,save_folder,df_loss = get_ds(modification=modification,args_init=args_init)
    model = full_model(ds, args).to(args.device)
    optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
    trainer = Trainer(ds,model,args,optimizer,loss_function,scheduler = scheduler,show_figure = False,trial_id = trial_id, fold=0,save_folder = save_folder)
    trainer.train_and_valid(normalizer = ds.normalizer, mod = 1000,mod_plot = None) 

else:
    (trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init = args_init,
                                                    station=station,modification=modification,
                                                    training_mode_to_visualise=training_mode_to_visualise)

----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 7392 elts between 2019-03-16 00:00:00 and 2019-05-31 23:45:00
Invalid dates within this fold: 776

>>>Tackle Target dataset: subway_in
   Load data from: /~/../../data/rrochas/prediction_validation/subway_in/subway_in.csv
   Init Dataset: 'torch.Size([7392, 40]). 0 Nan values
   TRAIN contextual_ds: torch.Size([2819, 40, 7])
   VALID contextual_ds: torch.Size([940, 40, 7])
   TEST contextual_ds: torch.Size([939, 40, 7])

>>>Tackle Contextual dataset:  subway_out
   Load data from: /~/../../data/rrochas/prediction_validation/subway_out/subway_out.csv
   Init Dataset: '[torch.Size([7392, 40])]. [tensor(0)] Nan values
   TRAIN contextual_ds: [torch.Size([2819, 40, 7])]
   VALID contextual_ds: [torch.Size([940, 40, 7])]
   TEST contextual_ds: [torch.Size([939, 40, 7])]
Init U/Utarget size: torch.Size([4699, 40, 7])/torch.Size([4699, 40, 4]) Train/Valid/Test 2819 940 939

------------

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Training Throughput:1476.37 sequences per seconds
>>> Training complete in: 0:02:20.694848
>>> Training (42.54%) performance time: min 0.01 avg 2.03e-02 seconds (+/- 3.28e-03)
>>> Loading (41.31%) performance time: min 0.00 avg 2.02e-02 seconds (+/- 9.27e-02)
>>> Forward  (26.18%) performance time: 1.30e-02 seconds (+/- 2.66e-03)
>>> Backward  (16.22%) performance time: 8.07e-03 seconds (+/- 1.89e-03)
>>> Plotting  (0.00%) performance time: 4.96e-06 seconds (+/- 1.03e-05)
>>> Saving  (16.29%) performance time: 7.78e-01 seconds (+/- 1.42e-01)
>>> PI-tracking  (0.00%) performance time: 7.71e-06 seconds (+/- 1.05e-05)
>>> Scheduler update  (0.00%) performance time: 6.27e-05 seconds (+/- 7.18e-05)
>>> Validation time: 0:00:00.394172

Max GPU memory allocated: 0.2780933380126953 GB
Max GPU memory cached: 0.388671875 GB
Max CPU memory allocated: 3.370441436767578 GB
None

--------------------------------------------------
Reload dataset without shuffling on train set, and remove data_augmen

Loading BokehJS ...

In [39]:
training_mode = 'test'
out_dim_factor = args.out_dim_factor 
df_verif = getattr(ds.tensor_limits_keeper,f"df_verif_{training_mode}")  
Preds,Y_true,_ = trainer.test_prediction()

In [60]:
nb_step_ahead = Preds.size(-1)//out_dim_factor
if stations is not None:
    remaining_column_ind = [k for k,c in enumerate(ds.spatial_unit) if c in stations]
else:
    stations = ds.spatial_unit
    remaining_column_ind = np.arange(len(ds.spatial_unit))

sub_Preds = Preds[:,remaining_column_ind,:].detach().cpu().numpy()
sub_Y_true = Y_true[:,remaining_column_ind,-1].detach().cpu().numpy()

def get_multi_index_columns(stations,L_out_dim_factor,L_nb_step_ahead):
    level_0,level_1,level_2 = [],[],[]
    for station in stations:
        for q in L_out_dim_factor:
            for h in L_nb_step_ahead:
                level_0.append(station)
                level_1.append(f'h{h+1}')
                level_2.append(f'q{q}')
    multi_index = pd.MultiIndex.from_tuples(list(zip(level_0,level_1,level_2)),names=['station','step_ahead','q'])
    return multi_index


df_true =  pd.DataFrame(sub_Y_true,columns = stations,index = df_verif.iloc[:,-1])

df_predictions = []
for i in range(out_dim_factor):
    for sa in range(nb_step_ahead):
        multi_index = get_multi_index_columns(stations,[i],[sa])
        df_predictions.append(pd.DataFrame(sub_Preds[:,:,nb_step_ahead*i + sa],
                                        columns = multi_index,
                                        index = df_verif.iloc[:,-nb_step_ahead+sa])
        )
#### MODIFIER DF-VERIF 
#### CHECKER LES APPEL DE DF-VERIF EN PARTICULIER POUR LA CORRESPONDANCE SUR X INPUT [-2]

In [68]:
df_predictions[3].columns[0]


('BEL', 'h4', 'q0')

In [77]:
ds.df_dates.shift(1)

,date
0,NaT
1,2019-03-16 00:00:00
2,2019-03-16 00:15:00
3,2019-03-16 00:30:00
4,2019-03-16 00:45:00
...,...
4593,2019-05-02 20:00:00
4594,2019-05-02 20:15:00
4595,2019-05-02 20:30:00
4596,2019-05-02 20:45:00


In [ ]:
legend_it = []
p = figure(x_axis_type="datetime", title= title,
                width=width,height=height)

if type(station) != list:
        station = [station]


""" New """

""" End New """ 

"""
for k,station_i in enumerate(station):
        c = p.line(x=df_true.index, line_width = 2.5, y=df_true[station_i], alpha=0.8,color = Plasma256[int(k*255/len(station))])
        legend_it.append((f'True_{station_i}', [c]))

        if df_prediction is not None: 
                if type(df_prediction) == list:
                    for q_i,df_pred in enumerate(df_prediction):
                            c = p.line(x=df_pred.index, line_width = 2.5, y=df_pred[station_i], alpha=0.6, line_dash = 'dashed',color = Plasma256[int(k*255/len(station))])
                            #legend_it.append((f'Prediction_{station_i}_q{q_i}', [c]))
                            legend_it.append((f'Pred_{station_i}', [c]))
                else :
                    c = p.line(x=df_prediction.index, line_width = 2.5, y=df_prediction[station_i], alpha=0.6, line_dash = 'dashed',color = Plasma256[int(k*255/len(station))])
                    legend_it.append((f'Pred_{station_i}', [c]))
"""

FileNotFoundError: [Errno 2] No such file or directory: '/home/rrochas/prediction-validation/~/../../'

In [18]:
L = [1,2,3,4]
L[:-1]

[1, 2, 3]

In [9]:
import pandas as pd 
DATE_COL = 'VAL_DATE'
LOCATION_COL = 'VAL_ARRET_CODE'
VALUE_COL = 'Flow'

START = '2019-11-01' # Exemple basé sur head()
END = '2020-04-30 23:30:00'
MIN_AVG_DAILY_PASSENGER = 24*10

dirname = '/home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle/tramway_indiv_15min/tramway_indiv_15min.csv'
df = pd.read_csv(dirname,index_col=0)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df['station_lane_sens'] = df['VAL_ARRET_CODE'].astype(str) + '_' + df['LIG_NUMERO_SAE'].astype(str) + '_' + df['CRS_SENS_TRAJET'].astype(str)
df = df.pivot_table(index=DATE_COL,columns='station_lane_sens',values=VALUE_COL)
reindex = pd.date_range(start=START, end=END, freq='15min')[:-1]
df = df.reindex(reindex).fillna(0)

# Filter useless stations: 
mask_init = df.resample('1D').sum().mean()>MIN_AVG_DAILY_PASSENGER
mask = mask_init[mask_init].index.to_list()
df = df[mask]

In [ ]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.benchmark import local_get_args
from examples.train_and_visu_non_recurrent import evaluate_config,train_the_config,get_ds
from high_level_DL_method import load_optimizer_and_scheduler
from dl_models.full_model import full_model
from trainer import Trainer

from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation,load_configuration,train_valid_1_model
import torch
save_folder = 'K_fold_validation/training_with_HP_tuning/subway_in_subway_out'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
args_init,folds = load_configuration(trial_id,True)

training_mode_to_visualise = ['test'] # ['test','valid','train']
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'

modification ={'keep_best_weights':True,
                'epochs':100,
                'device':torch.device("cuda:0"),
                'target_data':'subway_in',
                
                'use_target_as_context': False,
                'freq':'15min',
                'minmaxnorm':True,
                'standardize': False,
                'learnable_adj_matrix' : False,
                'data_augmentation': False,
                }

config_diffs = {}
config_diffs.update({'subway_in_subway_out':{'dataset_names':['subway_in','subway_out'],
                                        'data_augmentation': True,
                                        'DA_method': 'rich_interpolation',
                                        'stacked_contextual': True,
                                        'temporal_graph_transformer_encoder': False,
                                        'compute_node_attr_with_attn': False,
                                        }
                            })
for add_name_id,config_diff in config_diffs.items():
    config_diff.update(modification)

if False: 
    ds,args_ds,trial_id,save_folder,df_loss = get_ds(modification=modification,args_init=args_init)
    model = full_model(ds, args_ds).to(args_ds.device)
    optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args_ds)
    trainer = Trainer(ds,model,args_ds,optimizer,loss_function,scheduler = scheduler,show_figure = False,trial_id = trial_id, fold=0,save_folder = save_folder)
    trainer.train_and_valid(normalizer = ds.normalizer, mod = 1000,mod_plot = None) 
else:
    (trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init = args_init,
                                                    station=station,modification=modification,
                                                    training_mode_to_visualise=training_mode_to_visualise)




>>>> Load best CONFIG
----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 7392 elts between 2019-03-16 00:00:00 and 2019-05-31 23:45:00
Invalid dates within this fold: 776

>>>Tackle Target dataset: subway_in
   Load data from: /home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/subway_in/subway_in.csv
   Init Dataset: 'torch.Size([7392, 40]). 0 Nan values
   TRAIN contextual_ds: torch.Size([2821, 40, 7])
   VALID contextual_ds: torch.Size([940, 40, 7])
   TEST contextual_ds: torch.Size([940, 40, 7])

>>>Tackle Contextual dataset:  subway_out
   Load data from: /home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/subway_out/subway_out.csv
   Init Dataset: '[torch.Size([7392, 40])]. [tensor(0)] Nan values
   TRAIN contextual_ds: [torch.Size([2821, 40, 7])]
   VALID contextual_ds: [torch.Size([940, 40, 7])]
   TEST contextual_ds: [torch.Size([940, 40, 7])]
Init U/Utarget 

IndexError: list index out of range

In [7]:
vars(args_ds)

{'model_name': 'STGCN',
 'dataset_names': ['subway_in', 'subway_out'],
 'dataset_for_coverage': ['subway_in', 'netmob_POIs'],
 'device': device(type='cuda', index=0),
 'optimizer': 'adamw',
 'single_station': False,
 'loss_function_type': 'MSE',
 'epsilon_clustering': 0.05,
 'freq': '15min',
 'contextual_positions': {'calendar': [0, 1], 'subway_out': 2},
 'quick_vision': False,
 'netmob_transfer_mode': 'DL',
 'evaluate_complete_ds': True,
 'train_valid_test_split_method': 'similar_length_method',
 'set_spatial_units': None,
 'hp_tuning_on_first_fold': True,
 'keep_best_weights': True,
 'num_workers': 0,
 'persistent_workers': False,
 'pin_memory': True,
 'prefetch_factor': 2,
 'drop_last': False,
 'mixed_precision': False,
 'non_blocking': True,
 'torch_compile': False,
 'backend': 'inductor',
 'prefetch_all': False,
 'NetMob_selected_apps': ['Google_Maps', 'Deezer', 'Instagram'],
 'NetMob_transfer_mode': ['DL'],
 'NetMob_selected_tags': ['iris'],
 'NetMob_expanded': '',
 'ray': False,

In [ ]:
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args_ds)
trainer = Trainer(ds,model,args,optimizer,loss_function,scheduler = scheduler,show_figure = False,trial_id = trial_id, fold=0,save_folder = save_folder)
trainer.train_and_valid(normalizer = ds.normalizer, mod = 1000,mod_plot = None) 

'blocks': [[1], [32, 32, 32], [32, 32, 32], [64]]

In [5]:
vars(args)

{'model_name': 'STGCN',
 'dataset_names': ['subway_in', 'subway_out'],
 'dataset_for_coverage': ['subway_in', 'netmob_POIs'],
 'device': device(type='cuda'),
 'optimizer': 'adamw',
 'single_station': False,
 'loss_function_type': 'MSE',
 'epsilon_clustering': 0.05,
 'freq': '15min',
 'contextual_positions': {'subway_out': 2},
 'quick_vision': False,
 'netmob_transfer_mode': 'DL',
 'evaluate_complete_ds': True,
 'train_valid_test_split_method': 'similar_length_method',
 'set_spatial_units': None,
 'hp_tuning_on_first_fold': True,
 'keep_best_weights': False,
 'num_workers': 0,
 'persistent_workers': False,
 'pin_memory': True,
 'prefetch_factor': 2,
 'drop_last': False,
 'mixed_precision': False,
 'non_blocking': True,
 'torch_compile': False,
 'backend': 'inductor',
 'prefetch_all': False,
 'NetMob_selected_apps': ['Google_Maps', 'Deezer', 'Instagram'],
 'NetMob_transfer_mode': ['DL'],
 'NetMob_selected_tags': ['iris'],
 'NetMob_expanded': '',
 'ray': False,
 'ray_scheduler': 'ASHA',
 

In [35]:
os.listdir(dirname)

['METR_LA',
 'PEMS_BAY',
 'PEMS_d3',
 'PEMS_d4',
 'PEMS_d7',
 'data_bidon',
 'generate_data.ipynb',
 'netmob_bidon',
 'ref_subway.csv',
 'subway_in',
 'netmob_video_lyon',
 'netmob_image_per_station',
 'NetMob_lyon.geojson',
 'lyon_iris_shapefile',
 'POIs',
 'subway_out',
 'CRITER_3lanes',
 'agg_data']

In [11]:
contextual_train = {name_i: contextual_i['train'] for name_i,contextual_i in ds.contextual_tensors.items()}  
contextual_valid ={name_i: contextual_i['valid'] for name_i,contextual_i in ds.contextual_tensors.items() if 'valid' in contextual_i.keys()}  
contextual_test  = {name_i: contextual_i['test'] for name_i,contextual_i in ds.contextual_tensors.items() if 'test' in contextual_i.keys()}  

for name_i in contextual_train.keys():
    print(name_i,': ',contextual_train[name_i].size())

calendar_dayofweek :  torch.Size([971, 7])
calendar_hour :  torch.Size([971, 18])
subway_indiv :  torch.Size([834, 40, 7])


In [6]:
print('contextual_positions: ',ds.contextual_positions)
for contextual_i in ds.contextual_tensors.keys():
    print()
    for training_mode in ['train','valid','test']:
        print(f"contextual_tensors[{contextual_i}][{training_mode}]: ",ds.contextual_tensors[contextual_i][training_mode].size())

print('\nFirst/Last predicted train: ',ds.tensor_limits_keeper.first_predicted_train_date,ds.tensor_limits_keeper.last_predicted_train_date,'\n',
'First/Last predicted valid: ',ds.tensor_limits_keeper.first_predicted_valid_date,ds.tensor_limits_keeper.last_predicted_valid_date,'\n',
'First/Last predicted test: ',ds.tensor_limits_keeper.first_predicted_test_date,ds.tensor_limits_keeper.last_predicted_test_date
)
print('\ndf dates: ')
display(ds.tensor_limits_keeper.df_dates.sort_values(by='date'))
print('\ndf verif: ')
ds.tensor_limits_keeper.df_verif

output = next(iter(ds.dataloader['train']))

NameError: name 'ds' is not defined

In [9]:
import pandas as pd
dirname = '/home/rrochas/prediction-validation/../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle/subway_indiv_15min/'
os.listdir(dirname)
os.path.exists(f"{dirname}/subway_indiv_15min.csv")

FILE_BASE_NAME = 'subway_indiv'
DATA_SUBFOLDER = 'agg_data/validation_individuelle' # Sous-dossier dans FOLDER_PATH
NATIVE_FREQ = '3min'
START = '2019-10-01' # Exemple basé sur head()
END = '2020-04-01'
list_of_invalid_period = []
C = 1

DATE_COL = 'VAL_DATE'
LOCATION_COL = 'COD_TRG'
VALUE_COL = 'Flow'

target_freq = '15min'
file_name = f"{FILE_BASE_NAME}_{target_freq}"
data_file_path = f"{dirname}/{file_name}.csv"

df = pd.read_csv(data_file_path)
df['VAL_DATE'] = pd.to_datetime(df['VAL_DATE'])
df=df.set_index('VAL_DATE')
df.index.unique().sort_values()

DatetimeIndex(['2020-02-01 00:00:00', '2020-02-01 00:15:00',
               '2020-02-01 00:30:00', '2020-02-01 00:45:00',
               '2020-02-01 01:00:00', '2020-02-01 01:15:00',
               '2020-02-01 01:30:00', '2020-02-01 01:45:00',
               '2020-02-01 02:00:00', '2020-02-01 02:15:00',
               ...
               '2020-02-27 21:30:00', '2020-02-27 21:45:00',
               '2020-02-27 22:00:00', '2020-02-27 22:15:00',
               '2020-02-27 22:30:00', '2020-02-27 22:45:00',
               '2020-02-27 23:00:00', '2020-02-27 23:15:00',
               '2020-02-27 23:30:00', '2020-02-27 23:45:00'],
              dtype='datetime64[ns]', name='VAL_DATE', length=2262, freq=None)

In [ ]:
df

In [3]:
trainer.train_and_valid()


start training

Training Throughput:1063.21 sequences per seconds
>>> Training complete in: 0:04:49.253988
>>> Training performance time: min 0.016025543212890625 avg 0.028232812881469727 seconds (+/- 0.007409689556679389)
>>> Loading performance time: min 0.00011897087097167969 avg 0.021025142456797832 seconds (+/- 0.0533871588248097)
>>> Forward performance time: 0.014497923700226319 seconds (+/- 0.006785826679855182)
>>> Backward performance time: 0.01471411531366729 seconds (+/- 0.0026613273244625673)
>>> Plotting performance time: 2.2859429594260365e-06 seconds (+/- 7.815177432756738e-07)
>>> Saving performance time: 0.6629389921824137 seconds (+/- 0.29361714449532417)
>>> PI-tracking performance time: 5.977237643908016e-06 seconds (+/- 7.975377572890917e-06)
>>> Scheduler-update performance time: 4.9972054946362674e-06 seconds (+/- 9.795780319996475e-06)
>>> Validation time: 0:00:00.356755
Proportion of time consumed for Loading: 41.6%
Proportion of time consumed for Forward: 28

In [11]:
from scipy.sparse.linalg import eigs
import numpy as np 
import pandas as pd
L = (pd.DataFrame(np.random.rand(10,10))*10)


lambda_max = eigs(L.values , k=1, which='LR')[0].real

In [ ]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from trainer import Trainer
from high_level_DL_method import load_optimizer_and_scheduler
from dl_models.full_model import full_model

from plotting.plotting_bokeh import plot_bokeh


# Load config
model_name = 'STGCN' #'CNN'
dataset_names = ['subway_in','netmob']
args = get_args(model_name,dataset_names)

# Modification : 
args.K_fold = 5

args.ray = False
args.W = 0  # IMPORTANT AVEC NETMOB

args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

# optimization:
args.mixed_precision = True

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(args,folds)
K_subway_ds,args = K_fold_splitter.split_k_fold()
subway_ds = K_subway_ds[0]

ImportError: cannot import name 'match_period_coverage_with_netmob' from 'utils.utilities_DL' (/home/rrochas/prediction-validation/utils/utilities_DL.py)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [10]:
import torch

netmob_init = torch.Tensor([7392*4*263*287])
netmob_tensor = torch.Tensor([2934*4*263*287*8])

print('Init tensor: ',netmob_init)
print('Init feature vect: ',netmob_tensor)

print('\nblock1:')
sconv = torch.prod(torch.Tensor([64, 32, 131, 143, 8]))
tconv = torch.prod(torch.Tensor([64, 128, 131, 143, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)

print('\nblock2:')
sconv = torch.prod(torch.Tensor([64, 658, 65, 71, 8]))
tconv = torch.prod(torch.Tensor([64, 512, 65, 71, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)


Init tensor:  tensor([2.2318e+09])
Init feature vect:  tensor([7.0868e+09])

block1:
s_conv:  tensor(3.0692e+08) tconv:  tensor(1.2277e+09)

block2:
s_conv:  tensor(1.5548e+09) tconv:  tensor(1.2098e+09)
